In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
import os
import librosa

In [2]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# Load training and test datasets
train = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-3-alt/train/text.csv', encoding='ISO-8859-1')
test = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-3-alt/test/text.csv', encoding='ISO-8859-1')

# Preprocess text data function
def preprocess_text(text):
    # Remove special characters and lowercase
    text = re.sub(r'[^A-Za-z\s]', '', text.lower())
    # Tokenize and remove stopwords
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

# Apply preprocessing to both train and test data
train['Utterance'] = train['Utterance'].apply(preprocess_text)
test['Utterance'] = test['Utterance'].apply(preprocess_text)

# TF-IDF Vectorizer (fit on train data and transform train and test data)
vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the number of features as needed
X_train_tfidf = vectorizer.fit_transform(train['Utterance'])
X_test_tfidf = vectorizer.transform(test['Utterance'])  # Only transform the test data

# Encode target labels for train data
le = LabelEncoder()
y_train = le.fit_transform(train['Sentiment'])

# Split train data into training and validation sets (optional)
X_train, X_val, y_train, y_val = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

# Train the Random Forest Classifier model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Validate model performance on the validation set
y_pred_rf = rf_model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_pred_rf))
print("Classification Report:\n", classification_report(y_val, y_pred_rf))

# Predict sentiment for the test set
test_predictions_rf = rf_model.predict(X_test_tfidf)

# Convert the predictions back to original sentiment labels using LabelEncoder
test['Emotion'] = le.inverse_transform(test_predictions_rf)

# Check the predicted emotions for the test set
# print(test[['Utterance', 'Emotion']].head())

# Save the predictions to a CSV file for submission
# submission_df = pd.DataFrame({
#     'Sr No.': test['Sr No.'],
#     'Emotion': test['Emotion']
# })

# # Save the DataFrame to CSV
# submission_df.to_csv("submission.csv", index=False)


Validation Accuracy: 0.49
Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.39      0.40        61
           1       0.54      0.65      0.59        97
           2       0.44      0.26      0.33        42

    accuracy                           0.49       200
   macro avg       0.46      0.43      0.44       200
weighted avg       0.48      0.49      0.48       200



In [3]:
import pandas as pd

# Load training and test datasets
train_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-3-alt/train/text.csv', encoding='ISO-8859-1')
test_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-3-alt/test/text.csv', encoding='ISO-8859-1')

In [4]:
import os
from pydub import AudioSegment

# Define paths
video_folder = '/kaggle/input/ml-hackathon-ec-campus-set-3-alt/train/videos'
output_audio_folder = '/kaggle/working'

# Loop through each .mp4 file in the video folder
for filename in os.listdir(video_folder):
    if filename.endswith('.mp4'):
        video_path = os.path.join(video_folder, filename)
        audio_output_path = os.path.join(output_audio_folder, filename.replace('.mp4', '.wav'))
        
        try:
            # Load mp4 file
            audio = AudioSegment.from_file(video_path, format="mp4")
            
            # Export as .wav file
            audio.export(audio_output_path, format="wav")
            print(f"Converted {filename} to {audio_output_path}")
        
        except Exception as e:
            print(f"Error converting {filename}: {e}")

# Check the output directory to ensure the .wav files are created
print("Conversion complete.")

Converted dia43_utt6.mp4 to /kaggle/working/dia43_utt6.wav
Converted dia1036_utt15.mp4 to /kaggle/working/dia1036_utt15.wav
Converted dia395_utt5.mp4 to /kaggle/working/dia395_utt5.wav
Converted dia947_utt4.mp4 to /kaggle/working/dia947_utt4.wav
Converted dia1005_utt10.mp4 to /kaggle/working/dia1005_utt10.wav
Converted dia258_utt0.mp4 to /kaggle/working/dia258_utt0.wav
Converted dia14_utt2.mp4 to /kaggle/working/dia14_utt2.wav
Converted dia886_utt9.mp4 to /kaggle/working/dia886_utt9.wav
Converted dia38_utt2.mp4 to /kaggle/working/dia38_utt2.wav
Converted dia648_utt6.mp4 to /kaggle/working/dia648_utt6.wav
Converted dia246_utt1.mp4 to /kaggle/working/dia246_utt1.wav
Converted dia858_utt4.mp4 to /kaggle/working/dia858_utt4.wav
Converted dia130_utt16.mp4 to /kaggle/working/dia130_utt16.wav
Converted dia812_utt4.mp4 to /kaggle/working/dia812_utt4.wav
Converted dia46_utt14.mp4 to /kaggle/working/dia46_utt14.wav
Converted dia272_utt7.mp4 to /kaggle/working/dia272_utt7.wav
Converted dia750_utt

In [5]:
def extract_audio_features_enhanced(audio_path):
    """
    Extracts a variety of audio features from the audio file.
    """
    try:
        audio, sr = librosa.load(audio_path, sr=None)

        # MFCCs
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        mfccs_mean = np.mean(mfccs.T, axis=0)

        # Chroma
        chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
        chroma_mean = np.mean(chroma.T, axis=0)

        # Spectral Contrast
        spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
        spectral_contrast_mean = np.mean(spectral_contrast.T, axis=0)

        # Zero Crossing Rate
        zero_crossings = librosa.feature.zero_crossing_rate(y=audio)
        zero_crossing_mean = np.mean(zero_crossings)

        # Tempo
        tempo, _ = librosa.beat.beat_track(y=audio, sr=sr)

        # Combine all features into a single array
        features = np.hstack([mfccs_mean, chroma_mean, spectral_contrast_mean, zero_crossing_mean, tempo])
        return features
    except Exception as e:
        print(f"Error extracting features from {audio_path}: {e}")
        return None
    
    # Apply enhanced feature extraction
audio_features = []
audio_folder = "/kaggle/working"

for filename in os.listdir(audio_folder):
    if filename.endswith('.wav'):
        audio_path = os.path.join(audio_folder, filename)
        features = extract_audio_features_enhanced(audio_path)
        if features is not None:
            audio_features.append(features)

audio_features_df = pd.DataFrame(audio_features)
print(audio_features_df.head())

           0           1          2          3          4          5   \
0 -443.958466  161.514557 -44.486347  17.436211 -12.548977   0.480141   
1 -465.217163  145.629822 -40.965130  24.067696  -0.343997  -8.766479   
2 -384.635590  163.918121 -50.456795  39.099655   5.886077   6.008948   
3 -440.981842  144.218872 -64.646317  30.911291 -18.852642  11.822251   
4 -520.464478  141.688339 -13.649361  32.814419   8.091116   7.521530   

          6          7         8          9   ...        24         25  \
0  -7.370010  -5.816681 -4.132302 -13.616688  ...  0.427224  14.039572   
1  -3.334757 -19.234333 -4.414664  -8.216444  ...  0.338222  16.757154   
2  -4.733752  -6.155056  7.002968  -3.337719  ...  0.402780  13.711379   
3 -15.683924  -2.795408 -3.034352 -12.642783  ...  0.441310   9.246246   
4  14.749310  -0.265525 -1.889019  -4.195527  ...  0.481150  15.876309   

          26         27         28         29         30         31        32  \
0  10.386239  13.786767  14.930547 

In [6]:
from sklearn.preprocessing import StandardScaler

# Assuming audio_features_df already contains your audio features
scaler = StandardScaler()
audio_features_scaled = scaler.fit_transform(audio_features_df)

# Convert back to a DataFrame for easier manipulation
audio_features_scaled_df = pd.DataFrame(audio_features_scaled)

In [7]:
from sklearn.decomposition import PCA

# Applying PCA to reduce dimensionality
pca = PCA(n_components=20)  # Adjust the number of components as needed
audio_features_reduced = pca.fit_transform(audio_features_scaled_df)

# Convert to DataFrame
audio_features_reduced_df = pd.DataFrame(audio_features_reduced)

In [8]:
import pandas as pd

# Try specifying a different encoding, like 'ISO-8859-1' or 'latin1'
# train_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-1/train.csv', encoding='ISO-8859-1')

# Concatenate audio features with train_df (assuming order matches)
combined_train_df = pd.concat([train_df.reset_index(drop=True), audio_features_reduced_df], axis=1)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Separate features and labels
X = combined_train_df.drop(columns=['Sentiment', 'Sr No.', 'Utterance', 'Speaker', 'Dialogue_ID', 'Utterance_ID', 'Season', 'Episode', 'StartTime', 'EndTime'])

y = combined_train_df['Sentiment']

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train a Random Forest Classifier with tuned parameters
clf = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, class_weight='balanced')
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {val_accuracy:.2f}")
#print(classification_report(y_val, y_pred, target_names=label_encoder.classes_))

Validation Accuracy: 0.45


**The code generates a detailed log of the audio-text analysis process, including intermediate steps. and after this we have the done the video-text analysis**

In [10]:
import os
import shutil
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Function to clear the /kaggle/working/ directory
def clear_working_directory():
    for filename in os.listdir("/kaggle/working/"):
        file_path = os.path.join("/kaggle/working/", filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f"Error deleting file {file_path}: {e}")

# Function to convert .mp4 to .wav and store in /kaggle/working/
def convert_mp4_to_wav(mp4_folder, wav_folder):
    for filename in os.listdir(mp4_folder):
        if filename.endswith('.mp4'):
            mp4_path = os.path.join(mp4_folder, filename)
            wav_path = os.path.join(wav_folder, filename.replace('.mp4', '.wav'))
            try:
                os.system(f"ffmpeg -i {mp4_path} -ar 16000 {wav_path}")  # Change the sample rate if needed
                print(f"Converted {filename} to {wav_path}")
            except Exception as e:
                print(f"Error converting {filename}: {e}")

# Function to extract audio features
def extract_audio_features_enhanced(audio_path):
    try:
        audio, sr = librosa.load(audio_path, sr=None)
        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sr).T, axis=0)
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sr).T, axis=0)
        zero_crossing = np.mean(librosa.feature.zero_crossing_rate(y=audio))
        tempo, _ = librosa.beat.beat_track(y=audio, sr=sr)
        return np.hstack([mfccs, chroma, spectral_contrast, zero_crossing, tempo])
    except Exception as e:
        print(f"Error extracting features from {audio_path}: {e}")
        return None

# Clear the /kaggle/working/ directory
clear_working_directory()

# Convert .mp4 files to .wav and store in /kaggle/working/
audio_folder_test = "/kaggle/input/ml-hackathon-ec-campus-set-3-alt/test/videos"
convert_mp4_to_wav(audio_folder_test, "/kaggle/working/")

# Extract features from the new .wav files
audio_features_test = []
for filename in os.listdir("/kaggle/working/"):
    if filename.endswith('.wav'):
        audio_path = os.path.join("/kaggle/working/", filename)
        features = extract_audio_features_enhanced(audio_path)
        if features is not None:
            audio_features_test.append(features)

# Continue with feature scaling, dimensionality reduction, and predictions
audio_features_test_df = pd.DataFrame(audio_features_test)
audio_features_test_scaled = scaler.transform(audio_features_test_df)
audio_features_test_reduced = pca.transform(audio_features_test_scaled)
test_combined_df = pd.concat([test_df.reset_index(drop=True), pd.DataFrame(audio_features_test_reduced)], axis=1)
X_test = test_combined_df.drop(columns=['Utterance', 'Sr No.', 'Speaker', 'Dialogue_ID', 'Utterance_ID', 'Season', 'Episode', 'StartTime', 'EndTime'])
test_predictions = clf.predict(X_test)

# Prepare and save the submission file
submission_df = pd.DataFrame({
    'Sr No.': test_df['Sr No.'],
    'Emotion': label_encoder.inverse_transform(test_predictions)
})
submission_df.to_csv("/kaggle/working/submission.csv", index=False)


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Converted dia43_utt6.mp4 to /kaggle/working/dia43_utt6.wav
Converted dia158_utt1.mp4 to /kaggle/working/dia158_utt1.wav


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Converted dia70_utt0.mp4 to /kaggle/working/dia70_utt0.wav
Converted dia225_utt4.mp4 to /kaggle/working/dia225_utt4.wav


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

...

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Converted dia158_utt5.mp4 to /kaggle/working/dia158_utt5.wav
Converted dia182_utt9.mp4 to /kaggle/working/dia182_utt9.wav


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Converted dia167_utt4.mp4 to /kaggle/working/dia167_utt4.wav
Converted dia58_utt2.mp4 to /kaggle/working/dia58_utt2.wav


size=    1408kB time=00:00:07.50 bitrate=1536.3kbits/s speed= 162x    
video:0kB audio:1408kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.007075%


In [11]:
import os
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import re

# Load train and test datasets
train_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-3-alt/train/text.csv', encoding='ISO-8859-1')
test_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-3-alt/test/text.csv', encoding='ISO-8859-1')

# Define path to video clips
train_video_dir = '/kaggle/input/ml-hackathon-ec-campus-set-3-alt/train/videos'
test_video_dir = '/kaggle/input/ml-hackathon-ec-campus-set-3-alt/test/videos'

# Preprocessing function for text data
def preprocess_text(text):
    text = re.sub(r'[^A-Za-z\s]', '', text.lower())  # Lowercase and remove special characters
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(tokens)

# Apply text preprocessing to train and test data
train_df['Utterance'] = train_df['Utterance'].apply(preprocess_text)
test_df['Utterance'] = test_df['Utterance'].apply(preprocess_text)

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train_text = vectorizer.fit_transform(train_df['Utterance']).toarray()
X_test_text = vectorizer.transform(test_df['Utterance']).toarray()

# Function to get video file path from IDs
def get_video_clip_path(row, video_dir):
    dialogue_id = row['Dialogue_ID']
    utterance_id = row['Utterance_ID']
    filename = f"dia{dialogue_id}_utt{utterance_id}.mp4"
    return os.path.join(video_dir, filename)

# Apply function to get video paths
train_df['video_clip_path'] = train_df.apply(lambda row: get_video_clip_path(row, train_video_dir), axis=1)
test_df['video_clip_path'] = test_df.apply(lambda row: get_video_clip_path(row, test_video_dir), axis=1)

# Video feature extraction function
def extract_video_features(video_path, num_frames=5):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(1, frame_count // num_frames)

    for i in range(0, frame_count, interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (64, 64))  # Resize to reduce feature size
        frames.append(frame)
        if len(frames) >= num_frames:
            break

    cap.release()
    if len(frames) < num_frames:
        frames.extend([frames[-1]] * (num_frames - len(frames)))  # Padding if frames are less than num_frames

    return np.array(frames).flatten()

# Extract video features for train and test data
train_video_features = []
for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    train_video_features.append(extract_video_features(row['video_clip_path']))
train_video_features = np.array(train_video_features)

test_video_features = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    test_video_features.append(extract_video_features(row['video_clip_path']))
test_video_features = np.array(test_video_features)

# Combine text and video features
X_train = np.hstack((X_train_text, train_video_features))
X_test = np.hstack((X_test_text, test_video_features))

# Encode target labels for training
le = LabelEncoder()
y_train = le.fit_transform(train_df['Sentiment'])

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Validate model performance on the validation set
y_pred_rf = rf_model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_pred_rf))
print("Classification Report:\n", classification_report(y_val, y_pred_rf))

# Predict sentiment for the test set
test_predictions_rf = rf_model.predict(X_test)

# Convert predictions back to original sentiment labels
test_df['Emotion'] = le.inverse_transform(test_predictions_rf)

# Prepare submission file
submission_df = pd.DataFrame({
    'Sr No.': test_df['Sr No.'],
    'Emotion': test_df['Emotion']
})

# Save submission to CSV
submission_df.to_csv("submission.csv", index=False)


100%|██████████| 100/100 [00:36<00:00,  2.76it/s]


Validation Accuracy: 0.465
Classification Report:
               precision    recall  f1-score   support

           0       0.25      0.07      0.10        61
           1       0.50      0.91      0.64        97
           2       0.14      0.02      0.04        42

    accuracy                           0.47       200
   macro avg       0.30      0.33      0.26       200
weighted avg       0.35      0.47      0.35       200



till here only

In [12]:
# all_preds = ["your_prediction" for i in df['Utterance_ID']]
# all_ids = df["Sr No."]
# submission_df = pd.DataFrame({
#         'Sr No.': all_ids,
#         'Emotion': all_preds
#     })
    
# # Save the DataFrame to CSV
# submission_df.to_csv("submission.csv", index=False)